In [1]:
#!pip install numpy maptlotlib mpld3

In [2]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import sys
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [3]:
PROJECT_ROOT_DIR=Path("../../../").resolve()
assert PROJECT_ROOT_DIR.exists()
DATA_PATH = PROJECT_ROOT_DIR / 'data'
assert DATA_PATH.exists()
DATA_PATH_STRING = str(DATA_PATH)

In [4]:
# you need to compile the project with -DAARE_PYTHON_BINDINGS="ON"
sys.path.append(str((PROJECT_ROOT_DIR / 'build').resolve()))
# check if _aare can be imported
import importlib.util
_aare_module = importlib.util.find_spec("_aare")
assert _aare_module is not None


# File

In [5]:
from _aare import File

ImportError: To be able to register buffer protocol support for the type '_aare.Frame' the associated class<>(..) invocation must include the pybind11::buffer_protocol() annotation!

In [ ]:
# eiger json master file
file = File(DATA_PATH_STRING + "/eiger/eiger_500k_32bit_master_0.json")
print(file.rows,file.cols,file.bitdepth)
print(file.detector_type)
print(file.total_frames)
print("geometry",file.geometry)

In [ ]:
from _aare import DetectorType
list(DetectorType.__entries) # DetectorType is an enum

In [ ]:
# Frame class

In [ ]:
frame = file.read() # read frame from current position
frame

In [ ]:
frame.rows,frame.cols,frame.bitdepth,frame.size

In [ ]:
frame.array() # returns a numpy array of type unsigned int 

In [ ]:
file.iread(1) # read frame at index 1 (only complete frames are returned)
pos = file.tell() # get the index file is currently pointed to
print("pos",pos)
file.seek(0) # go to index 0
file.read(3) # read three frames. returns list of frames
# file.write(file.iread(0))

In [ ]:
# plot frame
plt.imshow(frame.array())
plt.show()

# ClusterFile

In [ ]:
from _aare import ClusterFileV2

In [ ]:
file = ClusterFileV2(DATA_PATH_STRING + "/clusters/beam_En700eV_-40deg_300V_10us_d0_f0_100.clust","r")
file

In [ ]:
file.read() # returns a list of the clusters of the current frame number

In [ ]:
file.read(2) # returns a list of clusters for two frames

In [ ]:
file.frame_number() # get the frame number where clusterFile currently points to

In [ ]:
clusters = file.read()
len(clusters)

In [ ]:
cluster = clusters[0]
print(cluster.frame_number, cluster.cluster.x,cluster.cluster.y) # get cluster information
np.array(cluster.cluster.data,np.int32) # view cluster data 

# Pedestal

In [ ]:
from _aare import Pedestal
file = File("/mnt/sls_det_storage/moench_data/testNewFW20230714/cu_half_speed_master_4.json")
file.rows,file.cols,file.detector_type

In [ ]:
pedestal = Pedestal(file.rows,file.cols)
pedestal.rows,pedestal.cols,pedestal.n_samples

In [ ]:
ndarray = pedestal.mean() # pedestal mean returns an internal NDArray object of type float8 and dimensions 2
ndarray

In [ ]:
np.array(ndarray) # to convert NDArray to numpy array with copying the data
np.array(ndarray,copy=False) # to convert NDArray to numpy array without copying the data


In [ ]:
ndarray.shape,ndarray.size,ndarray.bitdepth,ndarray.strides,ndarray.byte_strides

In [ ]:
ndarray[0,300]

In [ ]:
for i in range(2000):
    frame = file.iread(i)
    pedestal.push(frame) # push 2000 frames into the pedestal
    

In [ ]:
mean = np.array(pedestal.mean())
plt.imshow(mean)
plt.show()


In [ ]:
frame=file.iread(3000) # read frame at index 3000
plt.imshow(frame.array())
plt.title("Frame")
plt.show()

In [ ]:
frame=file.iread(3000) # read frame at index 3000
denoised_frame = frame.array()-pedestal.mean()
plt.imshow(denoised_frame)
plt.title("denoised frame")
plt.show()

In [ ]:
#other utilities in pedestal
pedestal.variance() # get NDArray of the variance for each pixel 
pedestal.standard_deviation() # get NDArray of the standard deviation for each pixel 
pedestal.sum # get NDArray of the sum squared for each pixel 
pedestal.sum2 # get NDArray of the sum squared for each pixel 
pedestal.clear() # resets the pedestal
# mean, variance, standard_deviation and clear can accept indexes to apply only specific pixels
row,col=0,0
pedestal.mean(row,col)
pedestal.variance(row,col)
pedestal.standard_deviation(row,col)
pedestal.clear(row,col)

# ClusterFinder

In [ ]:
from _aare import ClusterFinder

In [ ]:
cf = ClusterFinder(3,3,1.0,1) # cluster_sizeX, cluster_sizeY, nSigma, threshold (can be ignored depending on the function called later)

In [ ]:
clusters = cf.find_clusters_without_threshold(denoised_frame,pedestal)
len(clusters)

In [ ]:
# x_arr,y_arr=[],[]
# for cluster in clusters:
#     denoised_frame[cluster.x,cluster.y]=-1000
# plt.imshow(denoised_frame)
# plt.show()